In [9]:
import os
import pandas as pd
from dotenv import load_dotenv
load_dotenv()

from pulmoscanx.utils.dataset import ChestXrayDataset 
from pulmoscanx.utils.transform import train_transform, val_test_transform

In [6]:
ROOT_FOLDER = os.path.dirname(os.getcwd())
DATASET_DIR = os.path.join(ROOT_FOLDER, 'datasets')

test_path = os.path.join(DATASET_DIR, 'miccai2023_nih-cxr-lt_labels_test.csv')
train_path = os.path.join(DATASET_DIR, 'miccai2023_nih-cxr-lt_labels_train.csv')
val_path = os.path.join(DATASET_DIR, 'miccai2023_nih-cxr-lt_labels_val.csv')

train_dataset = ChestXrayDataset(
    csv_path=train_path,
    img_dir=os.path.join(DATASET_DIR, 'cxr', 'images'),
    transform=None)
val_dataset = ChestXrayDataset(
    csv_path=val_path,
    img_dir=os.path.join(DATASET_DIR, 'cxr', 'images'),
    transform=None)
test_dataset = ChestXrayDataset(
    csv_path=test_path,
    img_dir=os.path.join(DATASET_DIR, 'cxr', 'images'),
    transform=None)

In [13]:
from torch.utils.data import ConcatDataset, random_split, DataLoader

full_dataset = ConcatDataset([train_dataset, val_dataset, test_dataset])

total_size = len(full_dataset)
train_size = int(0.7 * total_size)
val_size = int(0.15 * total_size)
test_size = total_size - train_size - val_size

new_train_dataset, new_val_dataset, new_test_dataset = random_split(
    full_dataset, [train_size, val_size, test_size]
)

new_train_dataset.dataset.transform = train_transform
new_val_dataset.dataset.transform = val_test_transform
new_test_dataset.dataset.transform = val_test_transform

train_loader = DataLoader(new_train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(new_val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(new_test_dataset, batch_size=32, shuffle=False)